In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [10]:
def generate_sentence(max_len=20, pos=True):
    if pos:
        n = np.random.randint(np.ceil(max_len / 3))
        sentence = n*"a" + n*"b" + n*"c"
        return sentence, len(sentence)
    else:
        n_0 = np.random.randint(0, max_len)
        n_1 = np.random.randint(0, max_len - n_0 + 1)
        n_2 = np.random.randint(0, max_len - n_0 - n_1 + 1)
        sentence = n_0 * "a" + n_1 * "b" + n_2 * "c"
        return sentence, len(sentence)

def create_data(size=10000, balance=0.5):
    data = []
    sentence_lengths = []

    for i in range(int(size*balance)):
        sentence, sentence_length = generate_sentence(pos=True)
        data.append((sentence, 1))
        sentence_lengths.append(sentence_length)
    for i in range(int((size - (size*balance)))):
        sentence, sentence_length = generate_sentence(pos=False)
        data.append((sentence, 0))
        sentence_lengths.append(sentence_length)
    
    random.shuffle(data)
    average_length = sum(sentence_lengths) / len(sentence_lengths)
    
    return data, average_length

data, avg_sent_length = create_data()
print(f"Data:\n{data}")
print(f"Average Sentence Length:\n{avg_sent_length}")


Data:
[('aaabbbccc', 1), ('aaaaaaaaaabbbbbbbbbb', 0), ('aaaaaabbbbbbcccccc', 1), ('aaaabbbbcccc', 1), ('aaabbbbbcccccc', 0), ('', 1), ('aaaaabbbbbccccc', 1), ('aaabbbccc', 1), ('bbbbbbbbbbbbcc', 0), ('', 1), ('aabbcccccccc', 0), ('abc', 1), ('aaaabbbbbbccccccccc', 0), ('', 1), ('aaaaaabbbbbbcccccc', 1), ('', 1), ('aaaaabbbbbccccc', 1), ('', 1), ('aabbbbbbbbbbbbbbbbc', 0), ('abc', 1), ('aaabbbccc', 1), ('aaaaaaaaaaaaaaaaabc', 0), ('aaaaaaaaaaaaaaaaabb', 0), ('aaabbbccc', 1), ('aaabbbccc', 1), ('bbbbbcccccccccccc', 0), ('', 1), ('aaaaaaabbbbbccccc', 0), ('aaaaaabbbbbbcccccc', 1), ('aabbcc', 1), ('', 1), ('aaaaaaaaaaaaaaaaabcc', 0), ('aaaaabbbbbccccc', 1), ('aaaaaaaaaaaaaaaaaabb', 0), ('aaaabbbbbbbbbbbbbbbb', 0), ('aaaaaaaaaaaabbbbbb', 0), ('aaaaaaaaaaaaaaabbbcc', 0), ('aaaaabbbbbbbbbbb', 0), ('aaaaaabc', 0), ('aabbbbbbbbbbbbbccc', 0), ('aaaaaaaaaaaaaaaaaab', 0), ('aaaaaaaaaaaaaaaaa', 0), ('abbbbbbbbbbbbbbcccc', 0), ('aaaabbbbcccc', 1), ('aaaaaaaaaaaaaaaaaab', 0), ('aaaaaabbbbbbcccccc', 1